In [ ]:
%run "U1. Imports.ipynb"
%run "U2. Utilities.ipynb"

In [ ]:
# This reads in Chadwick register with player codes.
# Let's move this to Utilities and either make compatible with name_clean or make a separate one (mostly done in chadwick now)
keep_list = ['key_mlbam', 'key_fangraphs', 'key_bbref_minors', 'key_bbref', 'name_first', 'name_last']
chadwick = read_chadwick(keep_list)

# Take first two characters of first name
chadwick['First2'] = chadwick['name_first'].str.slice(0,2)
# And first 5 characters of last name
chadwick['Last5'] = chadwick['name_last'].str.slice(0,5)

# Make lower case
chadwick['First2'] = chadwick['First2'].str.lower()
chadwick['Last5'] = chadwick['Last5'].str.lower()

In [ ]:
# This reads in a map of team name, codes, and the number Fangraphs uses in their URLs
team_map = pd.read_csv(os.path.join(baseball_path, "Utilities", "Team Map.csv"))

# We just need teams right now
team_map = team_map[['FULLNAME', 'DKTEAM', 'BBREFTEAM', 'SFBBTEAM']]

In [ ]:
crosswalk = pd.read_csv(os.path.join(baseball_path, "Utilities", "Player Crosswalk.csv"), encoding='iso-8859-1') 
crosswalk = crosswalk[['mlbamid', 'steamerid']]
crosswalk.rename(columns={'mlbamid':'mlbamid_fill'}, inplace=True)

crosswalk.drop_duplicates('steamerid', keep='last', inplace=True)

### 1. Results

In [ ]:
%run "01. Results.ipynb"

In [ ]:
yesterdays_contests = get_yesterday()
get_history()

In [ ]:
print(yesterdays_contests)

### 2. Contests

In [ ]:
%run "02. Contests.ipynb"

In [ ]:
# Create contest list
contest = contests()
contest.to_csv(os.path.join(baseball_path, "2. Contests", "A. Daily Contests", f"Contests {todaysdate}.csv"), index=False)
### Compatibility with new format
contest['date'] = todaysdate
contest.to_csv(os.path.join(r'C:\Users\james\Documents\MLB\Database\A01. DraftKings\1. Contests', f'Contests {todaysdate}.csv'), index=False)

In [ ]:
# Identify today's contest you'll enter
todays_contests = daily_contests(todaysdate)
print(todays_contests)

In [ ]:
for i in range(len(todays_contests)):    
    contestName = todays_contests['Name'][i]
    print(contestName)
    
    if "Turbo" in contestName or "vs" in contestName or "Tiers" in contestName:
        continue
    
    contestKey = todays_contests['contestKey'][i]
    draftGroupId = todays_contests['draftGroupId'][i]
    
    # Payouts
    df = payouts(contestKey)
    df.to_csv(os.path.join(baseball_path, "2. Contests", "B. Payouts", "Payouts " + str(contestKey) + ".csv"))
        
    time.sleep(5)
        
    # Salaries
    salaries = draftables(draftGroupId)
    salaries.to_csv(os.path.join(baseball_path, "2. Contests", "C. Salaries", "Salaries " + str(contestKey) + ".csv"), encoding='iso-8859-1')  

    time.sleep(5)

### 3. MLB API

In [ ]:
%run "03. MLB API.ipynb"

In [ ]:
complete_df = create_statsapi("03/29/2023", "11/15/2023")
df = create_statcast("2023-03-15", "2023-11-15")

### 4. Dataset

In [ ]:
# This reads in a map of team name, codes, and the shorthand MLB uses in their URLs
team_map = pd.read_csv(os.path.join(baseball_path, "Utilities", "Team Map.csv"))

# We just need teams right now
team_map = team_map[['FULLNAME', 'BBREFTEAM', 'MLBURL', 'FANGRAPHSTEAM', 'VENUE_ID', 'SFBBTEAM', 'DKTEAM', 'ROTOWIRETEAM', 'FANPROSTEAM']]

In [ ]:
%run "04. Dataset.ipynb"

In [ ]:
# df = import_data(2019, 2023) # Changed to 2019 - keep full 2015- for training set below
sample = create_datasets(todaysdate)

### 6. Rosters

In [ ]:
%run "06. Rosters.ipynb"

In [ ]:
# List of games between dates
games = statsapi.schedule(start_date=yesterdaysdate_slash, end_date=yesterdaysdate_slash)

# Loop over games to create rosters
output = Parallel(n_jobs=-2, verbose=5)(delayed(create_rosters)(x=game, team_map=team_map) for game in games)

### 7. Stats

In [ ]:
%run "07. Stats.ipynb"

In [ ]:
create_team_rosters(yesterdaysdate)

### 9. Matchups

In [ ]:
%run "09. Matchups.ipynb"

In [ ]:
for contestKey in yesterdays_contests['contestKey']:
    try:
        print(contestKey)
        create_matchups(team_map, contestKey, date=yesterdaysdate, backtest=True, early=False)
    except:
        pass

### History

In [ ]:
%run "D1. History.ipynb"

### Projections

In [ ]:
%run "11. Projections.ipynb"

In [ ]:
roto_slates(todaysdate)

In [ ]:
dff_slates(todaysdate)

In [ ]:
%run "D2. Crosswalk.ipynb"

In [ ]:
xwalk = create_xwalk()